In [1]:
from aif360.sklearn.datasets import fetch_german

from numpy import mean
from numpy import std
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.metrics import fbeta_score, make_scorer, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from matplotlib import pyplot

pip install 'aif360[LawSchoolGPA]'


In [2]:
# load the dataset
def load_dataset():
    # load the dataset as a numpy array
    dataset = fetch_german()
    # split into inputs and outputs
    X, y = dataset.X, dataset.y

    # transform the age column into zero and one depending on the age being greater than 25
    X['age'] = X['age'].apply(lambda x: 0 if x <= 25 else 1)

    # transform the sex column into 0 or 1
    X['sex'] = X['sex'].apply(lambda x: 0 if x == 'female' else 1).astype(int)

    # select categorical features
    cat_ix = X.select_dtypes(include=['category']).columns
    num_ix = X.select_dtypes(include=['int64', 'float64']).columns
    # one hot encode cat features only
    # label encode the target variable to have the classes 0 and 1
    y = LabelEncoder().fit_transform(y)
    return X, y, cat_ix, num_ix

In [3]:
X, y, cat_ix, num_ix = load_dataset()

/home/dhasade/anaconda3/envs/audits/lib/python3.8/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [12]:
# Calculate the probabilities
prob_age_1 = X['age'].mean()  # P(age = 1)
prob_age_0 = 1 - prob_age_1   # P(age = 0)

prob_sex_1 = X['sex'].mean()  # P(sex = 1)
prob_sex_0 = 1 - prob_sex_1   # P(sex = 0)

print("P(age = 1):", prob_age_1)
print("P(age = 0):", prob_age_0)
print("P(sex = 1):", prob_sex_1)
print("P(sex = 0):", prob_sex_0)

P(age = 1): 0.81
P(age = 0): 0.18999999999999995
P(sex = 1): 0.69
P(sex = 0): 0.31000000000000005


In [14]:
# Calculate the conditional probabilities
prob_y_given_age_0 = y[X['age'] == 0].mean()  # P(y=1|age=0)
prob_y_given_age_1 = y[X['age'] == 1].mean()  # P(y=1|age=1)

prob_y_given_sex_0 = y[X['sex'] == 0].mean()  # P(y=1|sex=0)
prob_y_given_sex_1 = y[X['sex'] == 1].mean()  # P(y=1|sex=1)

print("P(y=1|age=0):", prob_y_given_age_0)
print("P(y=1|age=1):", prob_y_given_age_1)
print("P(y=1|sex=0):", prob_y_given_sex_0)
print("P(y=1|sex=1):", prob_y_given_sex_1)

P(y=1|age=0): 0.5789473684210527
P(y=1|age=1): 0.7283950617283951
P(y=1|sex=0): 0.6483870967741936
P(y=1|sex=1): 0.7231884057971014


In [15]:
# Calculate the joint probabilities
prob_y_given_sex_1_and_age_1 = y[(X['sex'] == 1) & (X['age'] == 1)].mean()  # P(y=1|sex=1, age=1)
prob_y_given_sex_1_and_age_0 = y[(X['sex'] == 1) & (X['age'] == 0)].mean()  # P(y=1|sex=1, age=0)
prob_y_given_sex_0_and_age_1 = y[(X['sex'] == 0) & (X['age'] == 1)].mean()  # P(y=1|sex=0, age=1)
prob_y_given_sex_0_and_age_0 = y[(X['sex'] == 0) & (X['age'] == 0)].mean()  # P(y=1|sex=0, age=0)

print("P(y=1|sex=1, age=1):", prob_y_given_sex_1_and_age_1)
print("P(y=1|sex=1, age=0):", prob_y_given_sex_1_and_age_0)
print("P(y=1|sex=0, age=1):", prob_y_given_sex_0_and_age_1)
print("P(y=1|sex=0, age=0):", prob_y_given_sex_0_and_age_0)


P(y=1|sex=1, age=1): 0.7388429752066116
P(y=1|sex=1, age=0): 0.611764705882353
P(y=1|sex=0, age=1): 0.697560975609756
P(y=1|sex=0, age=0): 0.5523809523809524


In [4]:
def demographic_parity(samples, y, attribute):
    # Calculate demographic parity for 'attribute'
    prob_y_given_attribute_1 = y[samples[attribute] == 1].mean()  # P(y=1|attribute=1)
    prob_y_given_attribute_0 = y[samples[attribute] == 0].mean()  # P(y=1|attribute=0)

    demographic_parity_attribute = abs(prob_y_given_attribute_1 - prob_y_given_attribute_0)

    return demographic_parity_attribute

In [5]:
# Demographic parity in the original dataset
demographic_parity_age = demographic_parity(X, y, 'age')
demographic_parity_sex = demographic_parity(X, y, 'sex')

print("Demographic Parity for 'sex':", demographic_parity_sex)
print("Demographic Parity for 'age':", demographic_parity_age)

Demographic Parity for 'sex': 0.07480130902290782
Demographic Parity for 'age': 0.14944769330734242


In [6]:
# split X and y into train, test and audit splits of 45%, 5% and 50% respectively
random_seed = 40
X_train, X_audit, y_train, y_audit = train_test_split(X, y, test_size=0.5, random_state=random_seed)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=random_seed)

In [7]:
# define models to test
def get_models():
    models, names = list(), list()
    # LR
    models.append(LogisticRegression(solver='liblinear'))
    names.append('LR')
    # LDA
    models.append(LinearDiscriminantAnalysis())
    names.append('LDA')
    # NB
    models.append(GaussianNB())
    names.append('NB')
    # GPC
    models.append(GaussianProcessClassifier())
    names.append('GPC')
    # SVM
    models.append(SVC(gamma='scale'))
    names.append('SVM')
    return models, names

models, names = get_models()

In [8]:
# define the preprocessing
ct = ColumnTransformer([('c',OneHotEncoder(),cat_ix), ('n',MinMaxScaler(),num_ix)])
X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

In [9]:
models, names = get_models()
best_acc = 0.0; best_model = None; best_model_name = None
for i in range(len(models)):

    model = models[i]
    model.fit(X_train, y_train)

    # evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{names[i]} Accuracy: {accuracy}")

    if accuracy > best_acc:
        best_acc = accuracy
        best_model = model
        best_model_name = names[i]

print(f"Best model: {best_model_name} with accuracy: {best_acc}")

LR Accuracy: 0.82
LDA Accuracy: 0.78
NB Accuracy: 0.5
GPC Accuracy: 0.72
SVM Accuracy: 0.78
Best model: LR with accuracy: 0.82


In [10]:
# Calculate the ground truth demographic parity of the best model
# This should be done on all data (train + test+ audit)
X_temp = ct.transform(X)
y_pred = best_model.predict(X_temp)

# Calculate demographic parity for 'age'
prob_y_given_age_1 = y_pred[X['age'] == 1].mean()  # P(y=1|age=1)
prob_y_given_age_0 = y_pred[X['age'] == 0].mean()  # P(y=1|age=0)

demographic_parity_age_of_model = abs(prob_y_given_age_1 - prob_y_given_age_0)
print("Demographic Parity for 'age':", demographic_parity_age_of_model)

# Calculate demographic parity for 'sex'
prob_y_given_sex_1 = y_pred[X['sex'] == 1].mean()  # P(y=1|sex=1)
prob_y_given_sex_0 = y_pred[X['sex'] == 0].mean()  # P(y=1|sex=0)

demographic_parity_sex_of_model = abs(prob_y_given_sex_1 - prob_y_given_sex_0)
print("Demographic Parity for 'sex':", demographic_parity_sex_of_model)

Demographic Parity for 'age': 0.22612085769980506
Demographic Parity for 'sex': 0.15521271622253385


In [11]:
def error_DP(samples, y, attribute, groud_truth_dp):
    # Calculate the error in demographic parity for 'attribute'
    return np.abs(groud_truth_dp - demographic_parity(samples, y, attribute))

In [14]:
ct_new = ColumnTransformer([('c',OneHotEncoder(),cat_ix), ('n',MinMaxScaler(),num_ix)])
ct_new.fit_transform(X_audit)

def BlackBox(samples):
    '''
    The black-box algorithm to audit
    samples : pandas.DataFrame
        The dataset.
    '''
    # transform using the same ColumnTransformer object used for the training data
    transformed_samples = ct_new.transform(samples)
    # predict
    yhat = best_model.predict(transformed_samples)

    # return yhat along with samples
    return samples, yhat

def RS(samples, n, attribute):
    '''
    Random Sampling.

    Parameters
    ----------
    samples : pandas.DataFrame
        The dataset.
    n : int
        The number of samples to be generated.
    attribute : str
        The attribute to be sampled.

    Attribute is not used in the method.
    '''

    subset = samples.sample(n=n, random_state=42)
    
    return BlackBox(subset)

In [19]:
samples, yhat = RS(X_audit, 25, 'age')

# calculate the error in demographic parity for 'age'
error_age = error_DP(samples, yhat, 'age')
print("Error in demographic parity for 'age':", error_age)

samples, yhat = RS(X_audit, 25, 'sex')

# calculate the error in demographic parity for 'sex'
error_sex = error_DP(samples, yhat, 'sex')
print("Error in demographic parity for 'sex':", error_sex)

Error in demographic parity for 'age': 0.35324422166527425
Error in demographic parity for 'sex': 0.1434885824787649
